In [20]:
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.cross_validation import train_test_split
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import f_classif


def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall==0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN

final_out_to_excel=[]
row0 = [u'特征集', u'样本个数', u'分类器', u'Accuracy', u'Precision', u'Recall', u'SN', u'SP',
                u'Gm', u'F_measure', u'F_score', u'MCC', u'ROC曲线面积', u'tp', u'fn', u'fp', u'tn']
final_out_to_excel.append(row0)

seq=[]
m6a_2614_sequence="/home01/chenhuangrong/ROC/m6a_data.txt"
RNA_code='ACGU'
# k=4
interval=2
gap=1
divided_num=10.0
division_num=10
fh=open(m6a_2614_sequence)
for line in fh:
    if line.startswith('>'):
        continue
    else:
        seq.append(line.replace('\n',''))
fh.close()

def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError
positive_seq=seq[:len(seq)/2]
negative_seq=seq[len(seq)/2:]

special_matrix=pd.read_csv('/home02/chenhuangrong/cluster_file.csv',header=None,index_col=False)
special_matrix_positive=special_matrix.iloc[:len(special_matrix)/2]
special_matrix_negative=special_matrix.iloc[len(special_matrix)/2:]
for select_num in xrange(1,204):
    y_pred_prob_all=[]
    y_pred_all=[]
    Y_all=[]
    ACC_all=0
    precision_all=0
    recall_all=0
    SN_all=0
    SP_all=0
    GM_all=0
    TP_all=0
    TN_all=0
    FP_all=0
    FN_all=0
    F_measure_all=0
    F1_Score_all=0
    pos_all=0
    neg_all=0
    MCC_all=0
    kf = KFold(n_splits=division_num,shuffle=True)  

    for train_index , test_index in kf.split(positive_seq):  
        positive_df=pd.DataFrame(positive_seq)
        train_special_matrix_positive=special_matrix_positive.iloc[train_index]
        test_special_matrix_positive=special_matrix_positive.iloc[test_index]
        positive_x_train=positive_df.iloc[train_index,:]
        positive_y_train=positive_df.iloc[test_index,:]
        negative_df=pd.DataFrame(negative_seq)
        train_special_matrix_negative=special_matrix_negative.iloc[train_index]
        test_special_matrix_negative=special_matrix_negative.iloc[test_index]
        negative_x_train=negative_df.iloc[train_index,:]
        negative_y_train=negative_df.iloc[test_index,:]

        train_special_matrix=pd.concat([train_special_matrix_positive,train_special_matrix_negative],axis=0)
        test_special_matrix=pd.concat([test_special_matrix_positive,test_special_matrix_negative],axis=0)




        positive_negative_x_train=pd.concat([positive_x_train,negative_x_train],axis=0)
        positive_negative_y_train=pd.concat([positive_y_train,negative_y_train],axis=0)
    # for interval in xrange(1,k+1):
        final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_x_train))]
        code_values=make_kmer_list(interval,RNA_code)
        code_len=len(code_values)
        positive_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
        negative_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
        for i,line_value in enumerate(positive_x_train.values):
            for j,code_value in enumerate(line_value[0]):
                if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                    for p,c_value in enumerate(code_values):
                        temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                        temp_value=[reduce(lambda x,y:x+y,temp_value)]
                        temp_value=temp_value[0] 

                        if c_value==temp_value:
                            positive_seq_value[p][j]+=1
        positive_seq_value=np.matrix(positive_seq_value)*1.0/(len(seq)/2)
        for i,line_value in enumerate(negative_x_train.values):
            for j,code_value in enumerate(line_value[0]):
                if j<= len(line_value[0])-interval-1-gap*(interval-1):
                    for p,c_value in enumerate(code_values):
                        temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                        temp_value=[reduce(lambda x,y:x+y,temp_value)]
                        temp_value=temp_value[0]
                        if c_value==temp_value:
                            negative_seq_value[p][j]+=1
        negative_seq_value=np.matrix(negative_seq_value)*1.0/(len(seq)/2)
        for i,line_value in enumerate(positive_negative_x_train.values):
            for j,code_value in enumerate(line_value[0]):
                if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                    for p,c_value in enumerate(code_values):
                        temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                        temp_value=[reduce(lambda x,y:x+y,temp_value)]
                        temp_value=temp_value[0]
                        if c_value==temp_value:
                              final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
        y_final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_y_train))]
        for i,line_value in enumerate(positive_negative_y_train.values):
            for j,code_value in enumerate(line_value[0]):
                if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                    for p,c_value in enumerate(code_values):
                        temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                        temp_value=[reduce(lambda x,y:x+y,temp_value)]
                        temp_value=temp_value[0]
                        if c_value==temp_value:
                              y_final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]

    #     final_seq_value1=[[0 for ii in xrange(len(seq[0])-1)] for jj in xrange(len(positive_negative_x_train))]
    #     code_values1=make_kmer_list(1,RNA_code)
    #     code_len1=len(code_values1)
    #     positive_seq_value1=[[0 for jj in xrange(len(seq[0])-1)] for ii in xrange(code_len1)]
    #     negative_seq_value1=[[0 for jj in xrange(len(seq[0])-1)] for ii in xrange(code_len1)]
    #     for i,line_value in enumerate(positive_x_train.values):
    #         for j,code_value in enumerate(line_value[0]):
    #             if j<= len(line_value[0])-1-1 :
    #                 for p,c_value in enumerate(code_values1):
    #                     if c_value==line_value[0][j:j+1]:
    #                         positive_seq_value1[p][j]+=1
    #     positive_seq_value1=np.matrix(positive_seq_value1)*1.0/(len(seq)/2)
    #     for i,line_value in enumerate(negative_x_train.values):
    #         for j,code_value in enumerate(line_value[0]):
    #             if j<= len(line_value[0])-1-1 :
    #                 for p,c_value in enumerate(code_values1):
    #                     if c_value==line_value[0][j:j+1]:
    #                         negative_seq_value1[p][j]+=1
    #     negative_seq_value1=np.matrix(negative_seq_value1)*1.0/(len(seq)/2)
    #     for i,line_value in enumerate(positive_negative_x_train.values):
    #         for j,code_value in enumerate(line_value[0]):
    #             if j<= len(line_value[0])-1-1 :
    #                 for p,c_value in enumerate(code_values1):
    #                     if c_value==line_value[0][j:j+1]:
    #                           final_seq_value1[i][j]=positive_seq_value1[p,j]-negative_seq_value1[p,j]
    #     y_final_seq_value1=[[0 for ii in xrange(len(seq[0])-1)] for jj in xrange(len(positive_negative_y_train))]


    #     for i,line_value in enumerate(positive_negative_y_train.values):
    #         for j,code_value in enumerate(line_value[0]):
    #             if j<= len(line_value[0])-1-1 :
    #                 for p,c_value in enumerate(code_values1):
    #                     if c_value==line_value[0][j:j+1]:
    #                           y_final_seq_value1[i][j]=positive_seq_value1[p,j]-negative_seq_value1[p,j]






    #     print len(y_final_seq_value),len(y_final_seq_value[0]),y_final_seq_value
    #     break
        X_train = np.array(final_seq_value)
        X_train1=np.array(train_special_matrix.values)
        X_train=np.concatenate((X_train,X_train1),axis=1)
        Y_train = list(map(lambda x: 1, xrange(len(final_seq_value) / 2)))
        Y2_train = list(map(lambda x: 0, xrange(len(final_seq_value) / 2)))
        Y_train.extend(Y2_train)
        Y_train = np.array(Y_train)
        F, pval = f_classif(X_train, Y_train)
        idx = np.argsort(F)
        selected_list_=idx[::-1]
        selected_list_=selected_list_[xrange(select_num)]
        X_train=pd.DataFrame(X_train)
        X_train=X_train.iloc[:,selected_list_]
        
        X_test = np.array(y_final_seq_value)
        X_test1=np.array(test_special_matrix.values)
        X_test=np.concatenate((X_test,X_test1),axis=1)
        X_test=pd.DataFrame(X_test)
        X_test=X_test.iloc[:,selected_list_]
        Y_test  = list(map(lambda x: 1, xrange(len(y_final_seq_value) / 2)))
        Y2_test  = list(map(lambda x: 0, xrange(len(y_final_seq_value) / 2)))
        Y_test.extend(Y2_test )
        Y_test  = np.array(Y_test)

        svc = svm.SVC(probability=True)
        parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
        clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
        clf.fit(X_train, Y_train)
        C=clf.best_params_['C']
        y_pred_prob=clf.predict_proba(X_test)

        gamma=clf.best_params_['gamma']
        print 'c:',C,'gamma:',gamma
        y_pred=clf.predict(X_test)

        y_pred_prob_all.extend(y_pred_prob)
        y_pred_all.extend(y_pred)
        Y_all.extend(Y_test)
        ACC=metrics.accuracy_score(Y_test,y_pred)
        print ACC
        precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_test, y_pred) 
        F1_Score=metrics.f1_score(Y_test, y_pred)
        F_measure=F1_Score
        MCC=metrics.matthews_corrcoef(Y_test, y_pred)

        pos=TP+FN
        neg=FP+TN
        ACC_all=ACC_all+ACC
        print "ACC_all:",ACC_all
        precision_all=precision_all+precision
        recall_all=recall_all+recall
        SN_all=SN_all+SN
        SP_all=SP_all+SP
        GM_all=GM_all+GM
        TP_all=TP_all+TP
        TN_all=TN_all+TN
        FP_all=FP_all+FP
        FN_all=FN_all+FN
        F_measure_all=F_measure_all+F_measure
        F1_Score_all=F1_Score_all+F1_Score
        pos_all=pos_all+pos
        neg_all=neg_all+neg
        MCC_all=MCC_all+MCC
    all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
    # pd.DataFrame(np.matrix(all_y).T).to_csv('/home01/chenhuangrong/ROC/PSNP_PSDNP_1gap_predict.csv',header=None,index=False)
    fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
    roc_auc = auc(fpr, tpr)
    savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
                SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
                FN_all,FP_all,TN_all,pos_all,neg_all]]]
    print savedata
    # easy_excel.save("svm_independent_test",[str(X_train.shape[1])],savedata,'/home01/chenhuangrong/ROC/PSNP_PSDNP_1gap.xls')
    final_out_to_excel.append(savedata)
pd.DataFrame(final_out_to_excel).to_excel("/home02/chenhuangrong/PsCPm6APred_PSNP_F_score.xlsx",sheet_name="F_score",index=False,header=False)

    
        



    

c: 0.25 gamma: 0.03125
0.5
ACC_all: 0.5
c: 0.25 gamma: 0.03125
0.5
ACC_all: 1.0
c: 0.25 gamma: 0.03125
0.5
ACC_all: 1.5
c: 0.25 gamma: 0.03125
0.5
ACC_all: 2.0
c: 0.25 gamma: 0.03125
0.5
ACC_all: 2.5
c: 0.25 gamma: 0.03125
0.5
ACC_all: 3.0
c: 0.25 gamma: 0.03125
0.5
ACC_all: 3.5
c: 0.25 gamma: 0.03125
0.5
ACC_all: 4.0
c: 0.25 gamma: 0.03125
0.5
ACC_all: 4.5
c: 0.25 gamma: 0.03125
0.5
ACC_all: 5.0
[[['svmC:0.25gamma:0.03125', 0.5, 0.5, 1.0, 1.0, 0.0, 0.0, 0.66666666666666674, 0.66666666666666674, 0.0, 0.5, 1307.0, 0.0, 1307.0, 0.0, 1307.0, 1307.0]]]
c: 14.2543794902 gamma: 0.793700525984
0.736641221374
ACC_all: 0.736641221374
c: 0.561231024155 gamma: 0.353553390593
0.717557251908
ACC_all: 1.45419847328
c: 2.82842712475 gamma: 0.157490131237
0.748091603053
ACC_all: 2.20229007634
c: 0.25 gamma: 0.353553390593
0.729007633588
ACC_all: 2.93129770992
c: 14.2543794902 gamma: 0.0701538780193
0.74427480916
ACC_all: 3.67557251908
c: 14.2543794902 gamma: 0.793700525984
0.702290076336
ACC_all: 4.37

IndexError: index 54 is out of bounds for axis 1 with size 54

In [25]:
final_out_to_excel=[]
row0 = [u'特征集', u'样本个数', u'分类器', u'Accuracy', u'Precision', u'Recall', u'SN', u'SP',
                u'Gm', u'F_measure', u'F_score', u'MCC', u'ROC曲线面积', u'tp', u'fn', u'fp', u'tn']
final_out_to_excel.append(row0)
ddd_data=['svmC:6.34960420787gamma:1.78179743628', 0.75441867293012332, 0.7622229897965412, 0.7414034057545508, 0.7414034057545508, 0.7674339401056958, 0.7537666808248048, 0.75099104867488564, 0.75099104867488564, 0.50989969223707554, 0.79961732745050629, 969.0, 338.0, 304.0, 1003.0, 1307.0, 1307.0]
final_out_to_excel.append(ddd_data)
pd.DataFrame(final_out_to_excel).to_excel("/home02/chenhuangrong/PSDNP_1gap_final_matrix_24demension.xlsx",sheet_name="F_score",index=False,header=False)